<a href="https://colab.research.google.com/github/hackerpranavpandey/DataBricks-LLM-Course---NLP-Tasks/blob/main/NLP_Task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Here in this notebook all NLP tasks of LLM are discussed and also implemented using Hugging face transformers, datasets and models** ##


Learning Objective
1. Use a variety of existing models for a variety of common applications
2. Understanding basic prompt engineering
3. Understanding search vs sampling for LLM interfaces.
4. Get familiar with the main Hugging Face abstractions : datasets, pipelines,takenization, and models.

In [ ]:
%pip install sacremoses==0.0.53

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 7.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895240 sha256=88298e657546385f2672d9c14adfe2b49ef46d2259aabd549606ef9066717db2
  Stored in directory: /root/.cache/pip/wheels/00/24/97/a2ea5324f36bc626e1ea0267f33db6aa80d157ee977e9e42fb
Successfully built sacremoses


In [ ]:
%pip install datasets ## needs to do this in order to use library dat
from datasets import load_dataset
from transformers import pipeline

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.7 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.


**Summarization can take two forms:**

1- **extractive** (select representative excerts from the text)

2- **abstractive** (generating novel text summaries)

Here we ar using model which does abstractive summarization.

Background Reading: Hugging face summarization page

We are using :-

Data: **xsum dataset**, which provides a set of BBC articles and summaries

Model: **t5-small model**, which has 60 millions parameters, developed by google is a encoder-decoder model which support several tasks such as summarization, translation,Q&A, and text classification.



In [ ]:
xsum_dataset=load_dataset(
    "xsum",
    version="1.2.0")
xsum_dataset

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1491: FutureWarning: The repository for xsum contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/xsum
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Generating train split:   0%|          | 0/204045 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11332 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11334 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 204045
    })
    validation: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 11332
    })
    test: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 11334
    })
})

This dataset provides 3 columns:

:- **documents** : BBC articles

:- **summary**: a ground-truth summary

:- **id **- article ID

In [ ]:
## Lets analyse some data
import numpy as np
import sys
print(sys.getsizeof(xsum_dataset["validation"])) ## gives size in terms of memory
print("Number of articles for training",np.size(xsum_dataset["train"]))
print("Number of articles for validation",np.size(xsum_dataset["validation"]))
print("Number of articles for testing",np.size(xsum_dataset["test"]))

48
Number of articles for training 204045
Number of articles for validation 11332
Number of articles for testing 11334


In [ ]:
## just using few among the train to use
xsum_samples= xsum_dataset["train"].select(range(100))
display(xsum_samples)   ## will not print them so need to convert then to first pandas
display(xsum_samples.to_pandas()[0:5])   ## use display instead of print

Dataset({
    features: ['document', 'summary', 'id'],
    num_rows: 100
})

,document,summary,id
0,"The full cost of damage in Newton Stewart, one...",Clean-up operations are continuing across the ...,35232142
1,A fire alarm went off at the Holiday Inn in Ho...,Two tourist buses have been destroyed by fire ...,40143035
2,Ferrari appeared in a position to challenge un...,Lewis Hamilton stormed to pole position at the...,35951548
3,"John Edward Bates, formerly of Spalding, Linco...",A former Lincolnshire Police officer carried o...,36266422
4,Patients and staff were evacuated from Cerahpa...,An armed man who locked himself into a room at...,38826984


We next use the Hugging Face pipeline tool to load a pre-trained model.In this LLM pipeline constructor, we specify:

**:-task:** The first argument specify the task . More are available on hugging fase

**:-model:** this is the name of model that that we want to use for our task like here 't5-small'

**:-min_lenght,max_length:** summarize the token between these two lenghts

**:-truncation:** truncate the input if needed since most of LLMs have limit on size of input but our article may go beyond the limit


In [ ]:
summarizer=pipeline(
    task="summarization",
    model="t5-small",
    min_length=10,
    max_length=50,
    truncation=True
)

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

In [ ]:
## Apply to 1 article
summarizer(xsum_samples["document"][0])

[{'summary_text': 'the full cost of damage in Newton Stewart is still being assessed . many roads in peeblesshire remain badly affected by standing water . the water breached a retaining wall, flooding many commercial properties .'}]

In [ ]:
# Apply to all the article
results=summarizer(xsum_samples["document"][0:10])

In [ ]:
## lets display generated one with actual summary
import pandas as pd
display(
    pd.DataFrame.from_dict(results)
    .rename({"summary_text": "generated_summary"}, axis=1)
    .join(pd.DataFrame.from_dict(xsum_samples))[
        ["generated_summary", "summary", "document"]
    ]
)

,generated_summary,summary,document
0,the full cost of damage in Newton Stewart is s...,Clean-up operations are continuing across the ...,"The full cost of damage in Newton Stewart, one..."
1,a fire alarm went off at the Holiday Inn in Ho...,Two tourist buses have been destroyed by fire ...,A fire alarm went off at the Holiday Inn in Ho...
2,Sebastian Vettel will start third ahead of tea...,Lewis Hamilton stormed to pole position at the...,Ferrari appeared in a position to challenge un...
3,the 67-year-old is accused of committing the o...,A former Lincolnshire Police officer carried o...,"John Edward Bates, formerly of Spalding, Linco..."
4,a man receiving psychiatric treatment at the c...,An armed man who locked himself into a room at...,Patients and staff were evacuated from Cerahpa...
5,Gregor Townsend gave a debut to powerhouse win...,Defending Pro12 champions Glasgow Warriors bag...,Simone Favaro got the crucial try with the las...
6,"Veronica Vanessa Chango-Alverez, 31, was kille...",A man with links to a car that was involved in...,"Veronica Vanessa Chango-Alverez, 31, was kille..."
7,the 25-year-old was hit by a motorbike during ...,Welsh cyclist Luke Rowe says changes to the sp...,Belgian cyclist Demoitie died after a collisio...
8,gundogan will not be fit for the start of the ...,Manchester City midfielder Ilkay Gundogan says...,"Gundogan, 26, told BBC Sport he ""can see the f..."
9,the crash happened about 07:20 GMT at the junc...,A jogger has been hit by an unmarked police ca...,The crash happened about 07:20 GMT at the junc...


Now Lets use sentiment analysis model

**data**:- poem sentiment is dataset with sentiment label for each of the poem

**model**- bold textbold textfine tuned version of BERT use for entity recognition too

In [ ]:
poem_dataset=load_dataset(
    "poem_sentiment",
    version="1.0.0"
)
print(poem_dataset)

/usr/local/lib/python3.10/dist-packages/datasets/load.py:1491: FutureWarning: The repository for poem_sentiment contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/poem_sentiment
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Generating train split:   0%|          | 0/892 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/105 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/104 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'verse_text', 'label'],
        num_rows: 892
    })
    validation: Dataset({
        features: ['id', 'verse_text', 'label'],
        num_rows: 105
    })
    test: Dataset({
        features: ['id', 'verse_text', 'label'],
        num_rows: 104
    })
})


In [ ]:
print("number of training dataset",np.size(poem_dataset["train"]))
print("number of validation dataset",np.size(poem_dataset["validation"]))
print("number of test",np.size(poem_dataset["test"]))
print("number of unique labels",np.unique(poem_dataset["train"]["label"]))

number of training dataset 892
number of validation dataset 105
number of test 104
number of unique labels [0 1 2 3]


In [ ]:
poem_samples=poem_dataset["train"].select(range(10))
display(poem_samples.to_pandas())

,id,verse_text,label
0,0,with pale blue berries. in these peaceful shad...,1
1,1,"it flows so long as falls the rain,",2
2,2,"and that is why, the lonesome day,",0
3,3,"when i peruse the conquered fame of heroes, an...",3
4,4,of inward strife for truth and liberty.,3
5,5,the red sword sealed their vows!,3
6,6,and very venus of a pipe.,2
7,7,"who the man, who, called a brother.",2
8,8,"and so on. then a worthless gaud or two,",0
9,9,to hide the orb of truth--and every throne,2


In [ ]:
sentiment_classifier=pipeline(
    task="text-classification",
    model="nickwong64/bert-base-uncased-poems-sentiment"
)

config.json:   0%|          | 0.00/923 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/348 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [ ]:
results=sentiment_classifier(poem_samples["verse_text"])

In [ ]:
result_df=pd.DataFrame.from_dict(results).rename({"label":"predicted_label"},axis=1)
poem_sample_df=pd.DataFrame.from_dict(poem_samples).rename({"label":"true_label"},axis=1)
joined_data=result_df.join(poem_sample_df)

In [ ]:
## change label indices to text label
sentiment_label={0:"negative",1:"positive",2:"no_impact",3:"mixed"}
joined_data=joined_data.replace({"true_label":sentiment_label})

In [ ]:
display(joined_data[["predicted_label","true_label","score","verse_text"]])

,predicted_label,true_label,score,verse_text
0,positive,positive,0.996594,with pale blue berries. in these peaceful shad...
1,no_impact,no_impact,0.998741,"it flows so long as falls the rain,"
2,negative,negative,0.995966,"and that is why, the lonesome day,"
3,mixed,mixed,0.968735,"when i peruse the conquered fame of heroes, an..."
4,mixed,mixed,0.975967,of inward strife for truth and liberty.
5,mixed,mixed,0.966580,the red sword sealed their vows!
6,no_impact,no_impact,0.998639,and very venus of a pipe.
7,no_impact,no_impact,0.998611,"who the man, who, called a brother."
8,negative,negative,0.996557,"and so on. then a worthless gaud or two,"
9,no_impact,no_impact,0.998519,to hide the orb of truth--and every throne


**Translation**

Some are for specific two languages and many more

:- **Data**: We will use some example hard-coded sentences.However, there are a variety of translation datasets availabel

:-**Models**:

**Helsinki-NLP/opus-mt-en-es** for English to spanish this model is based on Marian NMT, a neural machine translation framework developed by microsoft

t5-small as already used earlier as encoder-decoder model.

In [ ]:
from transformers import pipeline
from datasets import load_dataset

In [ ]:
en_to_es_translation=pipeline(
    task="translation",
    model="Helsinki-NLP/opus-mt-en-es"
)

config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/312M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/826k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.59M [00:00<?, ?B/s]

In [ ]:
en_to_es_translation("Hello guys, how are you")

[{'translation_text': 'Hola chicos, ¿cómo están?'}]

In [ ]:
t5_small_translation=pipeline(  ##t5-small is used for multiple languages
    task="text2text-generation",
    model="t5-small",
    max_length=100
)

In [ ]:
t5_small_translation("translate English to French: Hello , Welcome to IIT Dharwad")

[{'generated_text': 'Bonjour , Bienvenue à IIT Dharwad'}]

In [ ]:
t5_small_translation("translate English to French:Jai Hind")

[{'generated_text': 'Jai Hind'}]

**Zero-shot classification**

It is a task of classifying a piece of text into one of a few given categories without having label trained model.

**Data** - just using xsum dataset

**Model** - using nli-deberate-v3-small which is a fine tuned version of the DeBERTamodel.

In [ ]:
zero_shot_pipeline=pipeline(
    task="zero-shot-classification",
    model="cross-encoder/nli-deberta-v3-small"
)

config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/568M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/418 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/18.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:560: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [ ]:
import pandas as pd
def categorize_article(article):
  """
    This helper function defines the labels which model must use to label article.
  """
  results=zero_shot_pipeline(
      article,
      candidate_labels=["politics","finance","sports","science and technology"]
  )
  del results["sequence"]
  display(pd.DataFrame(results))

In [ ]:
categorize_article("""
The year was 2021 and the former young Sporting KC phenom donned the white shirt and navy shorts of the U.S. Men’s National Team. The U.S. went on to win the CONCACAF Gold Cup, and four days later, Busio commanded a record incoming transfer fee for Sporting KC as he was shipped to Venezia FC.
After three years in Italy, Busio, who turned 22 last month, is back in Kansas City to represent his country again. This time, he’s preparing for the 2024 Summer Olympics in Paris.
The national team’s pre-Olympic camp and other preparations will take place at the Compass Minerals National Performance Center in Kansas City, Kan. On Tuesday night, the USMNT’s under-23 squad will play Japan’s U-23s in a final tuneup for the Summer Games.
Unfortunately, Busio will not be able to play this week after sustaining an injury in Venezia’s final game of the season. He provided an assist on the goal that sent Venezia back to Serie A, the top division in Italian pro soccer.
“I don’t think I did the best recovery for (the injury) the last three days after the game,” Busio said during halftime of Sporting KC’s 2-1 victory over the Seattle Sounders on Saturday.
Busio said he’s relished being back in Kansas City. He’s staying with current Sporting KC player Jake Davis, who remains one of his closest friends.
Speaking of his time with Sporting KC, Busio offered a glimpse of how much he has grown since then. Not that he was ever really immature — even after signing a homegrown deal with KC at the tender age of 14, he exuded confidence.
But there’s something to be gained from experiencing the trials and tribulations of promotion and relegation in a country with a raging fever for the game of soccer — like Italy. Busio joined the Venezia team at 19 and received a quick dose of reality during his first season overseas.
Venezia struggled in Italy’s top flight, and at times Busio did, too. After one year in Serie A, Venezia was demoted to Serie B and found itself fighting to avoid back-to-back relegation the following season.
“You have that feeling, ‘We have to be good, right? We have to go right back up. We were just in the first division,’” Busio recalled. “It was the total opposite. It was still like a punch in the face.”
""")

,labels,scores
0,sports,0.494381
1,science and technology,0.197973
2,finance,0.165258
3,politics,0.142388


In [ ]:
categorize_article("""
In simplest terms, AI is computer software that mimics the ways that humans think in order to perform complex tasks, such as analyzing, reasoning, and learning. Machine learning, meanwhile, is a subset of AI that uses algorithms trained on data to produce models that can perform such complex tasks. Most AI is performed using machine learning, so th...
See full list on coursera.org,cricket""")

,labels,scores
0,science and technology,0.704966
1,sports,0.160775
2,finance,0.084545
3,politics,0.049714


**Few-shot learning**

Here, you give model an instruction, a few-query response example of how o follow that instruction, and then a new query


In [ ]:
few_shot_pipeline=pipeline(
    task="text-generation",
    model="EleutherAI/gpt-neo-1.3B",
    max_new_tokens=10
)

config.json:   0%|          | 0.00/1.35k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/5.31G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

In [ ]:
# get the token ID, which we will use as the end-of-sequence
eos_token_id = few_shot_pipeline.tokenizer.encode("###")[0]

In [ ]:
## without any example the model output is incorrect
result=few_shot_pipeline(
    """For each tweet,describe its sentiment:
    [Tweet]: "This new music video was incredible"
    [Sentiment]:""",
    eos_token_id=eos_token_id
)
print(result[0]["generated_text"])

Setting `pad_token_id` to `eos_token_id`:21017 for open-end generation.


For each tweet,describe its sentiment:
    [Tweet]: "This new music video was incredible"
    [Sentiment]: "This new music video was incredible"




In [ ]:
## with only 1 example the model may or my not get right answer
result=few_shot_pipeline(
     """For each tweet,describe its sentiment:
    [Tweet]: "This is the link to the article"
    [Sentiment]: Neutral
    ###
    [Tweet]: "This new music video was incredible"
    [Sentiment]:""",
    eos_token_id=eos_token_id
)

Setting `pad_token_id` to `eos_token_id`:21017 for open-end generation.


In [ ]:
print(result[0]["generated_text"])

For each tweet,describe its sentiment:
    [Tweet]: "This is the link to the article"
    [Sentiment]: Neutral
    ###
    [Tweet]: "This new music video was incredible"
    [Sentiment]: Neutral
    ###
   


In [ ]:
## now give one sentiment of each type
result=few_shot_pipeline(
    """For each tweet describe its sentiment

    [Tweet]:"I hate it when my phone battery id died"
    [Sentiment]:Negative

    ###

    [Tweet]:"I like Nationalist people to Hindu Rastra"
    [Sentiment]:Positive

    ###

    [Tweet]: "This is the link to the article"
    [Sentiment]: Neutral

    ###

    [Tweet]: "This new music video was incredible"
    [Sentiment]:""",

    eos_token_id=eos_token_id
)

Setting `pad_token_id` to `eos_token_id`:21017 for open-end generation.


In [ ]:
print(result[0]["generated_text"])

For each tweet describe its sentiment

    [Tweet]:"I hate it when my phone battery id died"
    [Sentiment]:Negative

    ###

    [Tweet]:"I like Nationalist people for Hindu Rastra"
    [Sentiment]:Positive

    ###

    [Tweet]: "This is the link to the article"
    [Sentiment]: Neutral

    ###

    [Tweet]: "This new music video was incredible"
    [Sentiment]:Positive

    ###

For


In [ ]:
## lets try some more things on election result
result=few_shot_pipeline(
    """ For each state in India give me name of leaders

    [State]:"Uttar Pradesh"
    [Leader]:"Yogi Adityanath"

    ###
    [State]:"Madhya Pradesh"
    [Leader]:"Shivraj Singh"

    ###

    [State]:"Jharkhand"
    [Leader]:""",
    eos_token_id=eos_token_id
)

Setting `pad_token_id` to `eos_token_id`:21017 for open-end generation.


In [ ]:
print(result[0]["generated_text"])

 For each state in India give me name of leaders

    [State]:"Uttar Pradesh"
    [Leader]:"Yogi Adityanath"

    ###
    [State]:"Madhya Pradesh"
    [Leader]:"Shivraj Singh"

    ###

    [State]:"Jharkhand"
    [Leader]:"Ram Chandra"

    ###



Prompt engineering is a new but critical technique for working with LLMs.Above were nice example of it

Hugging Face APIs

Search and sampling to generate text

Auto loaders for tokenizers and models

## Searching and sampling in inference

You may see parameters like num_beams,do_sample,etc. specified with hugging fce pipeline

Task of LLM is to predict next token,the next and so on.The goal is to generate hogh probability sequence of tokens,which is essentially a search


LLM uses two ways to search

Search:

  Greedy: Pick the songle next most likely token in search

  Beam Search: use parameter num_beams.

Sampling: Given the tokens generated so far pick the next token by sampling

  Top-K sampling: top_k modifies sampling parameter that picks k most likely tokens

  Top-p sampling: top_p modifies sampling  by limiting it to the most likely tokens up to probability mass p.

toggle between search and sampling via parameter do_sample

In [ ]:
## This is greedy search
## using summarizer
summarizer(xsum_samples["document"][0])

[{'summary_text': 'the full cost of damage in Newton Stewart is still being assessed . many roads in peeblesshire remain badly affected by standing water . the water breached a retaining wall, flooding many commercial properties .'}]

In [ ]:
## beam search
summarizer(xsum_samples["document"][0],num_beams=10)

[{'summary_text': 'the full cost of damage in Newton Stewart is still being assessed . many roads in peeblesshire remain badly affected by standing water . the water breached a retaining wall, flooding many commercial properties .'}]

In [ ]:
## Alternatively using sampling
summarizer(xsum_samples["document"][0],do_sample=True)

[{'summary_text': 'the full cost of damage in Newton Stewart is still being assessed . many roads in peeblesshire remain badly affected by standing water . a flood alert remains in place across the Borders due to constant rain .'}]

In [ ]:
## limit sampling to top_k and top_p probability
summarizer(xsum_samples["document"][0],do_sample=True,top_k=10,top_p=8)

[{'summary_text': 'the full cost of damage in Newton Stewart is still being assessed . many roads in peeblesshire remain badly affected by standing water . the water breached a retaining wall, flooding many commercial properties .'}]